<a href="https://colab.research.google.com/github/nirvanK11/Classification-of-Images-of-cats-and-dogs-Using-a-CNN/blob/main/classification_of_images_of_cats_and_dogs_deep_learning_pred_2_Sln.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification of Images of cats and dogs Using a CNN
(adapted from https://colab.research.google.com/github/mohnabil2020/machine_learning/blob/master/horse_or_human_classifier.ipynb#scrollTo=AwQbx0_vsDli)

##Uses images from a zip file

Download the zip files containg the images.  The training data ad the test data are stored in 2 separate zip files

In [ ]:
# load the libraries
import os
import zipfile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

Extract the data

In [ ]:
# extract the training data into a directory
local_zip = '/content/drive/MyDrive/dogs-vs-cats-vvsmall.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')



In [ ]:
# Directory with our training horse pictures for training
train_dog_dir = os.path.join('/content/dogs-vs-cats-vvsmall/train/dogs')

# Directory with our training human pictures for training
train_cat_dir = os.path.join('/content/dogs-vs-cats-vvsmall/train/cats')

# Directory with our training horse pictures for validation
validation_dog_dir = os.path.join('/content/dogs-vs-cats-vvsmall/validation/dogs')

# Directory with our training human pictures for validation
validation_cat_dir = os.path.join('/content/dogs-vs-cats-vvsmall/validation/cats')

Explore the data

In [ ]:
# print out the names of the first 10 files to give a nice feeling that the files are arranged as expected
# i.e. images of horses in the horses folder and images of humans in the human folder
train_dog_names = os.listdir(train_dog_dir)
print(train_dog_names[:10])

train_cat_names = os.listdir(train_cat_dir)
print(train_cat_names[:10])

validation_dog_names = os.listdir(validation_dog_dir)
print(validation_dog_names[:10])

validation_cat_names = os.listdir(validation_cat_dir)
print(validation_cat_names[:10])

In [ ]:
# print out the number of images in the directories
print('total training dog images:', len(os.listdir(train_dog_dir)))
print('total training cat images:', len(os.listdir(train_cat_dir)))
print('total validation dog images:', len(os.listdir(validation_dog_dir)))
print('total validation cat images:', len(os.listdir(validation_cat_dir)))

In [ ]:
# display images from the dataset
nrows = 6
ncols = 6
pic_index = 0

fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
# pick the first 8 horse pictures
next_dog_pix = [os.path.join(train_dog_dir, fname) 
                for fname in train_dog_names[pic_index-8:pic_index]]
# pick the first 8 human pictures
next_cat_pix = [os.path.join(train_cat_dir, fname) 
                for fname in train_cat_names[pic_index-8:pic_index]]

# add the horse and human pictures together in a list and plot in
# the specified number of rows and columns
for i, img_path in enumerate(next_dog_pix+next_cat_pix):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') 
  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()


Data Preparation.  Normalise the images and re-size them to the same size

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(300, 300, 3)))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dense(512, activation='relu'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory('/content/dogs-vs-cats-vvsmall/train',
                                                    target_size=(300, 300),
                                                    batch_size=32,
                                                    class_mode='binary')

In [ ]:
validation_generator = test_datagen.flow_from_directory('/content/dogs-vs-cats-vvsmall/train',
                                                        target_size=(300, 300),
                                                        batch_size=32,
                                                        class_mode='binary')

In [ ]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=60,
    validation_data=validation_generator,
    validation_steps=50,
    callbacks=[es])


In [ ]:
# summarize the history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

# summarize the history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()
